In [2]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")


In [3]:
import torch
from torch.utils.data import DataLoader, Dataset

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_idx = []
        self.target_idx = []
        token_ids = tokenizer.encode(txt)
        for i in range(0, len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_idx.append(torch.tensor(input_chunk))
            self.target_idx.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_idx)
    def __getitem__(self, index) :
        return self.input_idx[index], self.target_idx[index]



In [4]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
 stride=128, shuffle=True, drop_last=True,
 num_workers=0):
    
   tokenizer = tiktoken.get_encoding("gpt2")
   dataset   = GPTDatasetV1(txt, tokenizer, max_length, stride)
   dataloader = DataLoader(
     dataset,
     batch_size=batch_size,
     shuffle=shuffle,
     drop_last=drop_last,
     num_workers=num_workers
 )
   return dataloader

In [5]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs,targets = next(data_iter)
print(inputs)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])


#### Embedding word 

In [6]:
vocab_size = 50257 
output_dimensions= 256
max_length=4
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dimensions)


In [7]:
token_embeddings = token_embedding_layer(inputs)

In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length,output_dimensions)

print(pos_embedding_layer.weight[0:5])


tensor([[ 1.0183, -0.0332,  1.3903,  ...,  0.6729, -0.2223, -0.9441],
        [-0.1253,  0.3969, -2.3326,  ...,  1.3947, -0.4276, -0.0749],
        [-0.3946, -0.1935,  1.3070,  ..., -2.4497, -0.0785, -0.5367],
        [ 0.2382,  0.5946, -2.3808,  ..., -0.7303, -0.4566,  1.2787]],
       grad_fn=<SliceBackward0>)


In [11]:
posittion = pos_embedding_layer(torch.arange(context_length))
input_embeddings = token_embeddings + posittion
print(input_embeddings.shape)


torch.Size([8, 4, 256])
